## Ceación de ofertas para UNR

* Inicialmente iniciamos la BD de SFE para hallar el PPP para saber el G de compra de SFE

In [16]:
from db_utils import fetch_general_table

df = fetch_general_table()
df.head()

/Users/penguinblack97/scripts/UNR/db_utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,contrato,periodo,tarifa,tarifa_indexada,GWh,Nombre,Tipo
0,C86125,2025-01-01,300.0,310.11,0.258,URRAO,Compra
1,C79905,2025-01-01,410.0,429.03,0.200,TERPEL,Venta
2,C78232,2025-01-01,410.0,429.03,0.480,TERPEL,Venta
3,C78228,2025-01-01,410.0,429.03,0.500,TERPEL,Venta
4,C78924,2025-01-01,306.0,325.21,0.375,NOVA,Compra


In [17]:
## Averiguo primero los valores de lo que se tiene contratado
import pandas as pd
df_filtrado = df[df["Tipo"] == "Compra"]
df_filtrado['año'] = df_filtrado['periodo'].dt.year
promedio_ponderado = (
    df_filtrado.groupby('año')
    .apply(lambda x: round((x['GWh'] * x['tarifa']).sum() / x['GWh'].sum(),2))
    .reset_index(name='PPP')
)
promedio_ponderado

/var/folders/yv/ddlrydjn4vb1xqknnq1zlzq00000gn/T/ipykernel_5093/3300605479.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['año'] = df_filtrado['periodo'].dt.year
/var/folders/yv/ddlrydjn4vb1xqknnq1zlzq00000gn/T/ipykernel_5093/3300605479.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: round((x['GWh'] * x['tarifa']).sum() / x['GWh'].sum(),2))


,año,PPP
0,2025,331.65
1,2026,324.97
2,2027,322.84
3,2028,320.13
4,2029,315.90
5,2030,312.44
6,2031,309.11
7,2032,307.62
8,2033,305.83
9,2034,301.86


#### Analisis de la curva de cliente 

* Teniendo en cuenta la curva tipica del cliente se halla el coeficiente de variación y relativo con reglas del 30%

In [18]:
df_demanda = pd.read_csv("matriz_unr.csv")

In [19]:
media = df_demanda['Consumo_kWh'].mean()
desviacion = df_demanda['Consumo_kWh'].std()
maximo = df_demanda['Consumo_kWh'].max()
minimo = df_demanda['Consumo_kWh'].min()

print(f"Promedio (kWh): {round(media, 2)}")
print(f"Desviación estándar (kWh): {round(desviacion, 2)}")
print(f"Máximo (kWh): {maximo}")
print(f"Mínimo (kWh): {minimo}")

Promedio (kWh): 705.83
Desviación estándar (kWh): 187.82
Máximo (kWh): 940
Mínimo (kWh): 400


In [20]:
cv = desviacion/media
print(f"Coeficiente de variación (%): {round(cv, 2)}")
cr = (maximo - minimo)/media
print(f"Coeficiente relativo: {round(cr, 2)}")

Coeficiente de variación (%): 0.27
Coeficiente relativo: 0.77


In [21]:
#### Hallo el perfil que tiene mi usuario
def clasificar_perfil(cv, cr):
    """
    Clasifica el tipo de curva energética según el Coeficiente de Variación (cv)
    y el Coeficiente Relativo (cr).

    Reglas:
    - Si CV y CR son 0 → 'Plano'
    - Si uno de los dos es menor a 0.3 → 'Piso'
    - Si ambos son mayores a 0.3 → 'Techo'
    """
    if cv == 0 and cr == 0:
        return "Plano"
    elif cv < 0.3 or cr < 0.3:
        return "Piso"
    elif cv > 0.3 and cr > 0.3:
        return "Techo"
    else:
        return "Indefinido"
#### Los años que yo quiero que mi usuario se contrate
def filtrar_periodo_contrato(df, año_inicial, duracion_contrato):
    año_final = año_inicial + duracion_contrato - 1
    df.rename(columns={"PPP": "PPP G base"},inplace=True)
    df_filtrado = df[(df['año'] >= año_inicial) & (df['año'] <= año_final)]
    return df_filtrado


### Creación de oferta y descuento por fidelidad - volumen

In [22]:
### Año de inicio de contrato 2025 y 5 años de contrato
duracion = 4
año = 2026
##########################################################################################
df_base = filtrar_periodo_contrato(promedio_ponderado, año, duracion)
df_base

,año,PPP G base
1,2026,324.97
2,2027,322.84
3,2028,320.13
4,2029,315.90


In [23]:
#### Hallo los AOM actuales de SFEC
def agregar_columna_ipc(df, ipc_base=0.0482, incremento_extra=0.05, valor_inicial=290, nombre_columna="Gastos SFEC"):
    """
    Agrega una columna al DataFrame que aumenta cada año según IPC + incremento adicional para los gastos de SFEC.

    Parámetros:
    - df: DataFrame con columna 'año'
    - ipc_base: IPC anual estimado (ej: 0.0482 para 4.82%)
    - incremento_extra: Porcentaje adicional (ej: 0.05 para 5%)
    - valor_inicial: Valor base para el primer año
    - nombre_columna: Nombre de la nueva columna

    Retorna:
    - DataFrame con la nueva columna agregada
    """
    tasa_total = ipc_base + incremento_extra
    años = df['año'].sort_values().unique()
    ### Se le suma (2) dos pesos para tener un valor agregado de costos SIC y CND 
    valores = {año: round((valor_inicial) * ((1 + tasa_total) ** i), 2) for i, año in enumerate(años)}
    df[nombre_columna] = df['año'].map(valores)
    df['AOM+SIC'] = round(df[nombre_columna]*12/df['GWh'],2)+2
    return df
gastos=agregar_columna_ipc(df_filtrado.groupby("año")[['GWh']].sum().reset_index())

In [24]:
df_unido = pd.merge(df_base, gastos, on='año', how='inner')

columnas_deseadas = ['año', 'PPP G base', 'AOM+SIC']
df_G = df_unido[columnas_deseadas]
df_G ["PPP G compra"]=round(df_G["PPP G base"] + df_G["AOM+SIC"],2)

/var/folders/yv/ddlrydjn4vb1xqknnq1zlzq00000gn/T/ipykernel_5093/2044428553.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_G ["PPP G compra"]=round(df_G["PPP G base"] + df_G["AOM+SIC"],2)


In [25]:
def aplicar_ajuste(df, columna_valores, tipo_curva):
    """
    Aplica ajustes económicos sobre la columna seleccionada según el tipo de curva.

    Reglas:
    - Si tipo_curva es 'Piso': se aplica 10% de ganancia → valor * 1.10
    - Si es 'Plano': se descuenta 1% sobre el resultado que tendría 'Piso' → valor * 1.10 * 0.99
    - Si es 'Techo': se incrementa 1% sobre el resultado que tendría 'Piso' → valor * 1.10 * 1.01

    Parámetros:
    - df: DataFrame que contiene la columna de valores
    - columna_valores: str, nombre de la columna a ajustar
    - tipo_curva: str, puede ser 'Piso', 'Plano' o 'Techo'

    Retorna:
    - DataFrame con una nueva columna 'Ajuste económico'
    """

    # Valor base ajustado por tipo
    if tipo_curva == "Piso":
        factor = 1.10
    elif tipo_curva == "Plano":
        factor = 1.10 * 0.99  # descuento del 1% sobre el piso
    elif tipo_curva == "Techo":
        factor = 1.10 * 1.01  # recargo del 1% sobre el piso
    else:
        raise ValueError("Tipo de curva inválido. Usa 'Piso', 'Plano' o 'Techo'.")

    # Aplicar el ajuste a la columna
    df["PPP G venta"] = round(df[columna_valores] * factor,2)

    return df
df_ajustado = aplicar_ajuste(df_G, "PPP G compra", tipo_curva=clasificar_perfil(cv, cr))
print("Tipo de curva",clasificar_perfil(cv,cr))
print(df_ajustado.head())


Tipo de curva Piso
    año  PPP G base  AOM+SIC  PPP G compra  PPP G venta
0  2026      324.97    22.41        347.38       382.12
1  2027      322.84    17.00        339.84       373.82
2  2028      320.13    18.48        338.61       372.47
3  2029      315.90    19.69        335.59       369.15


In [26]:
import pandas as pd

def generar_tasas_fidelidad_volumen(años=range(3, 11), volumenes=[55, 110, 220, 440, 880], tasa_fidelidad_base=0.0015, tasa_volumen_base=0.0010):
    """
    Crea un DataFrame con tasas de fidelidad y volumen que crecen proporcionalmente.

    - Fidelidad: de 0% en año 3 y voy creciendo de 0.15% por año.
    - Volumen: se duplica por cada rango, empezando en 0% para el primer.

    Retorna:
    - DataFrame con columnas: 'Año', 'Tasa Fidelidad (%)', 'Rango Volumen', 'Tasa Volumen (%)'
    """
    filas = []
    años_lista = list(años)
    total_periodos = len(años_lista) - 1  # 7 para años 3–10

    for i, año in enumerate(años_lista):
        tasa_fidelidad = round((i ) * tasa_fidelidad_base, 5) if i > 0 else 0

        for j, vol_min in enumerate(volumenes):
            vol_max = volumenes[j + 1] if j + 1 < len(volumenes) else "∞"
            rango = f"{vol_min}-{vol_max}" if vol_max != "∞" else f">{vol_min}"
            tasa_volumen = round(tasa_volumen_base * (j) if j > 0 else 0, 5)

            filas.append({
                "Año": año,
                "Tasa Fidelidad (%)": round(tasa_fidelidad * 100, 2),
                "Rango Volumen": rango,
                "Tasa Volumen (%)": round(tasa_volumen * 100, 2)
            })

    return pd.DataFrame(filas)
df_tasas = generar_tasas_fidelidad_volumen()

In [27]:
df_tasas["Tasa Total (%)"] = round(df_tasas["Tasa Fidelidad (%)"] + df_tasas["Tasa Volumen (%)"],2)

In [28]:
fila_seleccionada = df_tasas[(df_tasas["Año"] == duracion) & (df_tasas["Rango Volumen"] == "220-440")]
print("Tasa de descuento por fidelidad y volumen de contrato")
print(fila_seleccionada)
df_ajustado["Tasa de descuento (%)"]=fila_seleccionada["Tasa Total (%)"].values[0]

Tasa de descuento por fidelidad y volumen de contrato
   Año  Tasa Fidelidad (%) Rango Volumen  Tasa Volumen (%)  Tasa Total (%)
7    4                0.15       220-440               0.2            0.35


In [29]:
df_ajustado["PPP G venta con descuento"] =round((100-df_ajustado['Tasa de descuento (%)'])*df_ajustado['PPP G venta']/100,2)

In [30]:
df_ajustado

,año,PPP G base,AOM+SIC,PPP G compra,PPP G venta,Tasa de descuento (%),PPP G venta con descuento
0,2026,324.97,22.41,347.38,382.12,0.35,380.78
1,2027,322.84,17.00,339.84,373.82,0.35,372.51
2,2028,320.13,18.48,338.61,372.47,0.35,371.17
3,2029,315.90,19.69,335.59,369.15,0.35,367.86
